# Module 2: Row vs. Column Stores
**Goal**: Shatter the illusion that "Data is Data."
The physical layout of bytes on a disk determines whether your database is a Ferrari or a Dump Truck. In this chapter, we explore the two fundamental religions of database physics: **Row-Oriented** (The General Practitioner) and **Column-Oriented** (The Specialist).
- **Row Stores (Postgres, MySQL)**: Keep all data for one person together.
- **Column Stores (DuckDB, Snowflake, Parquet)**: Keep all data for one attribute together.

---

## 1. Setup and Data Loading
First, we establish connections to our two laboratory subjects:
1. **Postgres**: Represents the traditional Row Store (OLTP).
2. **DuckDB**: Represents the modern Column Store (OLAP).

We will load the `users.csv` and `orders_sorted.csv` datasets into both engines to ensure a fair fight.

In [ ]:
import duckdb
import psycopg2
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# Configure Visualization
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [10, 5]

# --- 1. Connect to DBs ---
pg_conn = psycopg2.connect(host="db_int_opt", user="admin", password="password", dbname="db_int_opt")
pg_cursor = pg_conn.cursor()
duck_conn = duckdb.connect()

# --- 2. The Smart Loader Function ---
def smart_load_to_postgres(file_path, table_name):
    """
    Uses DuckDB to infer the CSV schema, then generates the Postgres CREATE TABLE 
    statement dynamically. This prevents 'Schema Mismatch' errors.
    """
    print(f"🕵️‍♂️ Inspecting {table_name} schema via DuckDB...")
    
    # Use DuckDB to read the file and get data types
    # We load 1 row just to get the columns
    df_schema = duck_conn.execute(f"SELECT * FROM read_csv_auto('{file_path}') LIMIT 1").df()
    
    # Map Pandas/DuckDB types to Postgres Types
    col_defs = []
    for col, dtype in df_schema.dtypes.items():
        pg_type = "TEXT" # Default
        if "int" in str(dtype): pg_type = "BIGINT"
        elif "float" in str(dtype): pg_type = "FLOAT"
        elif "datetime" in str(dtype): pg_type = "TIMESTAMP"
        elif "object" in str(dtype) and "date" in col: pg_type = "DATE" # Heuristic for dates
        elif "bool" in str(dtype): pg_type = "BOOLEAN"
        
        col_defs.append(f'"{col}" {pg_type}') # Quote cols to handle reserved words
    
    create_sql = f"CREATE TABLE {table_name} ({', '.join(col_defs)});"
    
    # Execute in Postgres
    pg_cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
    pg_cursor.execute(create_sql)
    
    # Load Data
    with open(file_path, 'r') as f:
        next(f) # Skip header
        pg_cursor.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV", f)
    pg_conn.commit()
    print(f"✅ Loaded {table_name} successfully into Postgres.")

# --- 3. Execute Loading ---
print("⏳ Starting Robust Data Load...")
users_file = "../data/users.csv"
orders_file = "../data/orders_sorted.csv"

try:
    # Load Postgres (Row Store)
    smart_load_to_postgres(users_file, "users")
    smart_load_to_postgres(orders_file, "orders")
    
    # Load DuckDB (Column Store)
    duck_conn.execute(f"CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv_auto('{users_file}')")
    duck_conn.execute(f"CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv_auto('{orders_file}')")
    print("🎉 All Data Loaded Successfully.")
    
except Exception as e:
    pg_conn.rollback()
    print(f"❌ Critical Error: {e}")

---

## Experiment 2.1: The OLTP Model (The Card Catalog)
**The Concept**: In a **Row Store**, data is stored like a card catalog or a Rolodex. If you want to know everything about "User 101", the database grabs one "card" (Page) and has the ID, Name, Email, and Status right there in a single contiguous block of memory.

In a **Column Store**, that same user's data is scattered across different memory regions. To rebuild the "User 101" object, the database must jump to the "ID List", then the "Name List", then the "Email List" and stitch them together (Tuple Reconstruction).

**Hypothesis**: Postgres (Row) should be faster (or at least highly competitive) at fetching a single, complete record compared to DuckDB (Column).

In [ ]:
import time

# --- Step 1: Define the Query (Fetch a single entity) ---
# We use a random ID. We need to check the actual column name for ID from the load above.
# Based on typical generation, it's likely 'user_id' or 'id'.
id_col = "user_id" 
target_id = 5000  

query = f"SELECT * FROM users WHERE {id_col} = {target_id}"

# --- Step 2: Measure Postgres (Row Store) ---
start_time = time.time()
for _ in range(100): # Run 100 times to average out noise
    pg_cursor.execute(query)
    result = pg_cursor.fetchone()
pg_duration = (time.time() - start_time) / 100

# --- Step 3: Measure DuckDB (Column Store) ---
start_time = time.time()
for _ in range(100):
    duck_conn.execute(query).fetchone()
duck_duration = (time.time() - start_time) / 100

# --- Step 4: Visualize ---
print(f"Postgres (Row) Time: {pg_duration:.6f}s")
print(f"DuckDB (Col) Time:   {duck_duration:.6f}s")

plt.figure(figsize=(8, 4))
sns.barplot(x=['Postgres (Row Store)', 'DuckDB (Column Store)'], y=[pg_duration, duck_duration], palette="viridis")
plt.title("Time to Fetch ONE Complete Row (SELECT * WHERE ID=X)")
plt.ylabel("Execution Time (Seconds)")
plt.show()

### Observation & Conclusion: The "Embedded" Anomaly
Your graph likely shows that **DuckDB** (Column Store) was significantly faster, which contradicts the classic textbook rule that Row Stores are better for single-row lookups.

**Why did the "wrong" database win?** You have just discovered the cost of **Network Latency vs. In-Process Execution**.
- **The Hidden Cost (Postgres)**: Postgres runs as a separate Service (Client-Server). Even though it found the row instantly on disk, it had to package that data, send it over a TCP network port (to Python), and wait for Python to unpack it. That "commute" took 90% of the time.
- **The In-Memory Advantage (DuckDB)**: DuckDB runs Embedded inside your Python process. It has zero network overhead. It grabs the data directly from RAM.
- **The Physics is still true**: DuckDB did have to perform Tuple Reconstruction (stitching the Name, Email, and ID vectors back together), which is inefficient. However, the penalty for that was tiny compared to the "Network Tax" Postgres paid.

**Key Takeaway**: In a local lab, architecture (Embedded vs. Server) often outweighs storage physics (Row vs. Column). If this were a real cloud app with both DBs on a remote server, Postgres would likely win this test.

---

## Experiment 2.2: The OLAP Model (The Specialist)
**The Concept**: We want to perform a heavy math operation over the entire dataset.
- **Row Store (Postgres)**: Must read every row (ID, User, Product, Date) just to extract the `quantity` integer. It reads mostly "waste."
- **Column Store (DuckDB)**: Reads *only* the `quantity` column. It ignores the IDs and Dates entirely.

**Hypothesis**: DuckDB should significantly outperform Postgres because it touches less data on the disk/memory.

In [ ]:
# --- Experiment 2.2: The OLAP Model (Corrected) ---
import time

# 1. Define the Query (Math over the whole dataset)
# We use "quantity" because we now know it exists!
col_name = "quantity"
query = f'SELECT AVG({col_name}) FROM orders'

# 2. Measure Postgres (Row Store)
start_time = time.time()
for _ in range(10): # Run 10 times to smooth out variance
    pg_cursor.execute(query)
    pg_cursor.fetchone()
pg_agg_duration = (time.time() - start_time) / 10

# 3. Measure DuckDB (Column Store)
start_time = time.time()
for _ in range(10):
    duck_conn.execute(query).fetchone()
duck_agg_duration = (time.time() - start_time) / 10

# 4. Visualize
print(f"Postgres (Row) Time: {pg_agg_duration:.6f}s")
print(f"DuckDB (Col) Time:   {duck_agg_duration:.6f}s")

# Calculate Speedup 
speedup = pg_agg_duration / duck_agg_duration if duck_agg_duration > 0 else 0

plt.figure(figsize=(8, 4))
sns.barplot(x=['Postgres (Row Store)', 'DuckDB (Column Store)'], y=[pg_agg_duration, duck_agg_duration], palette="magma")
plt.title(f"Time to Calculate AVG(quantity) - {speedup:.1f}x Speedup")
plt.ylabel("Execution Time (Seconds)")
plt.show()

### Observation
You should see a clear win for DuckDB here.
- **Why?** DuckDB loaded a single contiguous array of integers (`quantity`) and used CPU Vector instructions (SIMD) to sum them up. Postgres had to jump through row headers and pointers for every single record.

---

## Experiment 2.3: Compression (The Squeeze)
**The Concept**: Column stores compress better because data of the same type is stored together.
- **Users Table**: We previously verified (via `users` file inspection) that `account_status` exists. This is a low-cardinality text column (e.g., mostly "Active" or "Inactive"), which is perfect for **Run-Length Encoding (RLE)**.

**Hypothesis**: The Columnar Parquet file will be drastically smaller than the Row-based CSV file.

In [ ]:
import os

# --- Step 1: Prepare the Data ---
# We grab the 'users' table which we successfully loaded earlier
df = duck_conn.execute("SELECT * FROM users ORDER BY account_status").df()

# --- Step 2: Write as CSV (Row-oriented, uncompressed text) ---
csv_path = "../data/experiment_users.csv"
df.to_csv(csv_path, index=False)

# --- Step 3: Write as Parquet (Column-oriented, Snappy compressed) ---
parquet_path = "../data/experiment_users.parquet"
df.to_parquet(parquet_path, engine='pyarrow', compression='snappy')

# --- Step 4: Measure Sizes ---
csv_size = os.path.getsize(csv_path) / 1024 / 1024 # MB
parquet_size = os.path.getsize(parquet_path) / 1024 / 1024 # MB

# --- Step 5: Visualize ---
print(f"CSV Size:     {csv_size:.2f} MB")
print(f"Parquet Size: {parquet_size:.2f} MB")
reduction = (1 - (parquet_size / csv_size)) * 100

plt.figure(figsize=(8, 4))
sns.barplot(x=['CSV (Row)', 'Parquet (Column)'], y=[csv_size, parquet_size], palette="Blues_d")
plt.title(f"Storage Footprint: {reduction:.1f}% Reduction")
plt.ylabel("File Size (MB)")
plt.show()

### Observation & Conclusion
The Parquet file is likely greater than 50% smaller than the CSV.
- **Homogeneity**: Compression algorithms love repetition. In a Column store, the `account_status` column looks like `['Active', 'Active', 'Active'...]`. This compresses to almost nothing.
- **Type Awareness**: The database knows the `age` column is Integers. It doesn't store them as text characters ("2", "5"), but as tight binary integers.

**Final Lesson**: Use **Row Stores** (Postgres) when you are building an app that manages individual entities (Users, Orders, Carts). Use **Column Stores** (DuckDB, Snowflake) when you are analyzing trends across populations.